## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-27-21-50-00 +0000,wsj,Opinion | What the Iranian Regime Does Best,https://www.wsj.com/opinion/iran-irgc-australi...
1,2025-08-27-21-47-35 +0000,startribune,Timeline of mass shooting at Minneapolis church,https://www.startribune.com/timeline-annunciat...
2,2025-08-27-21-45-06 +0000,nyt,Former Puerto Rico Governor Pleads Guilty to C...,https://www.nytimes.com/2025/08/27/us/puerto-r...
3,2025-08-27-21-43-00 +0000,wsj,"Susan Monarez, the director of the Centers for...",https://www.wsj.com/politics/policy/cdc-direct...
4,2025-08-27-21-42-55 +0000,wapo,Children’s Minnesota Hospital-Minneapolis said...,https://www.washingtonpost.com


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,trump,60
122,new,17
5,shooting,15
60,gaza,14
49,school,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
321,2025-08-27-00-11-00 +0000,wsj,President Trump’s attempt to fire Fed governor...,https://www.wsj.com/politics/policy/trumps-fed...,144
338,2025-08-26-22-42-00 +0000,wsj,President Trump’s attempt to fire Federal Rese...,https://www.wsj.com/economy/central-banking/un...,135
3,2025-08-27-21-43-00 +0000,wsj,"Susan Monarez, the director of the Centers for...",https://www.wsj.com/politics/policy/cdc-direct...,111
133,2025-08-27-16-15-57 +0000,nypost,Cracker Barrel changed logo after call with Wh...,https://nypost.com/2025/08/27/business/cracker...,109
176,2025-08-27-13-51-00 +0000,nypost,Trump administration announces it’s going to r...,https://nypost.com/2025/08/27/us-news/trump-ad...,106


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
321,144,2025-08-27-00-11-00 +0000,wsj,President Trump’s attempt to fire Fed governor...,https://www.wsj.com/politics/policy/trumps-fed...
94,82,2025-08-27-18-34-05 +0000,nypost,Mom of Minneapolis mass shooter Robin Westman ...,https://nypost.com/2025/08/27/us-news/mom-of-m...
4,53,2025-08-27-21-42-55 +0000,wapo,Children’s Minnesota Hospital-Minneapolis said...,https://www.washingtonpost.com
41,49,2025-08-27-21-01-42 +0000,nypost,Angry Cracker Barrel workers dish on skimpy pa...,https://nypost.com/2025/08/27/business/angry-c...
176,46,2025-08-27-13-51-00 +0000,nypost,Trump administration announces it’s going to r...,https://nypost.com/2025/08/27/us-news/trump-ad...
228,45,2025-08-27-10-00-00 +0000,nypost,Grassley insists to ‘Pod Force One’ that Biden...,https://nypost.com/2025/08/27/us-news/grassley...
207,42,2025-08-27-11-15-50 +0000,cbc,Denmark summons top U.S. diplomat after report...,https://www.cbc.ca/news/world/denmark-us-green...
170,40,2025-08-27-14-30-00 +0000,wsj,A Beijing-linked espionage campaign that hit U...,https://www.wsj.com/politics/national-security...
310,39,2025-08-27-01-00-00 +0000,wsj,The first Black woman to serve as a Federal Re...,https://www.wsj.com/economy/central-banking/li...
272,36,2025-08-27-07-04-00 +0000,wsj,Israeli Troops Targeted a Camera in Gaza Hospi...,https://www.wsj.com/world/middle-east/israeli-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
